# **CTE**

> - **Temp named result set (virtual table) that can be used multipke times within your query to simplify and organize your query**
> - **CTES can be reused**
> - **Modularity , Readability , Reusablity**
> 
> - **CTE types** 
> 
>     - **Recursive**
>     - ****Non Recursive****
>         - ****Standalone**** 
>         - ****nested****

In [1]:
select * from SalesDB.Sales.Orders


(10 rows affected)

Total execution time: 00:00:00.060

OrderID,ProductID,CustomerID,SalesPersonID,OrderDate,ShipDate,OrderStatus,ShipAddress,BillAddress,Quantity,Sales,CreationTime
1,101,2,3,2025-01-01,2025-01-05,Delivered,9833 Mt. Dias Blv.,1226 Shoe St.,1,10,2025-01-01 12:34:56.0000000
2,102,3,3,2025-01-05,2025-01-10,Shipped,250 Race Court,NULL,1,15,2025-01-05 23:22:04.0000000
3,101,1,5,2025-01-10,2025-01-25,Delivered,8157 W. Book,8157 W. Book,2,20,2025-01-10 18:24:08.0000000
4,105,1,3,2025-01-20,2025-01-25,Shipped,5724 Victory Lane,,2,60,2025-01-20 05:50:33.0000000
5,104,2,5,2025-02-01,2025-02-05,Delivered,NULL,NULL,1,25,2025-02-01 14:02:41.0000000
6,104,3,5,2025-02-05,2025-02-10,Delivered,1792 Belmont Rd.,NULL,2,50,2025-02-06 15:34:57.0000000
7,102,1,1,2025-02-15,2025-02-27,Delivered,136 Balboa Court,,2,30,2025-02-16 06:22:01.0000000
8,101,4,3,2025-02-18,2025-02-27,Shipped,2947 Vine Lane,4311 Clay Rd,3,90,2025-02-18 10:45:22.0000000
9,101,2,3,2025-03-10,2025-03-15,Shipped,3768 Door Way,,2,20,2025-03-10 12:59:04.0000000
10,102,3,5,2025-03-15,2025-03-20,Shipped,NULL,NULL,0,60,2025-03-16 23:25:15.0000000


# **Standalone CTE**

> **independent cte , not dependency with other ctes**
> 
> **Syntax:**
> 
> **with cte\_name AS(query)**
> 
> **we cannot have order by in CTE's**

In [2]:
with cte_total_sales as 
(
    select 
        CustomerID
        ,sum(Sales) as TotalSales
    from SalesDB.Sales.Orders
    group by CustomerID
),
cte_customer_last_orderd_date as
(
    select
        CustomerID,
        Max(OrderDate) last_order_date
    from SalesDB.Sales.Orders 
    group by CustomerID
),
cte_customer_rank as
(
    select 
        *,
        RANK() OVER(order by TotalSales desc) as customer_rank 
    from cte_total_sales
),
cte_customer_segment as
(
    select 
        *,
        case 
            when TotalSales > 100 then 'high'
            when TotalSales > 50 then 'medium'
            else 'low'
        end as segment
    from cte_total_sales
)
select c.* , ts.TotalSales , lo.last_order_date , cr.customer_rank , cs.TotalSales
from SalesDB.Sales.Customers c
left join cte_total_sales ts on ts.CustomerID = c.CustomerID
left join cte_customer_last_orderd_date lo on lo.CustomerID = c.CustomerID
left join cte_customer_rank cr on cr.CustomerID = c.CustomerID
left join cte_customer_segment cs on cs.CustomerID = c.CustomerID
where ts.TotalSales is not NULL
order by customer_rank;

(4 rows affected)

Total execution time: 00:00:00.026

CustomerID,FirstName,LastName,Country,Score,TotalSales,last_order_date,customer_rank,TotalSales
3,Mary,NULL,USA,750,125,2025-03-15,1,125
1,Jossef,Goldberg,Germany,350,110,2025-02-15,2,110
4,Mark,Schwarz,Germany,500,90,2025-02-18,3,90
2,Kevin,Brown,USA,900,55,2025-03-10,4,55


# **Recursive CTE**

**it is executed until a condition is met**

> **with cte\_name as** 
> 
> **(**
> 
> **(select query) -- anchor query starting point of the tierations**
> 
> **union all**
> 
> **(select** 
> 
> **query** **from same\_cte\_name** 
> 
> **where break condition) -- recursive query executes multiple times**
> 
> **)**

In [10]:
-- sequence number from 1 to 20

with series as(
   (select 1 as myNumber) -- anchor query 
   
    union all 
    
    (select myNumber+1
    from series 
    where myNumber < 30) -- recursive query -- DEFULT IS 100 SET MAX REQUIRED RECURSION IN (MAXRECURSION LIMIT)
)

select * from series
OPTION(MAXRECURSION 100)
;

(30 rows affected)

Total execution time: 00:00:00.023

myNumber
1
2
3
4
5
6
7
8
9
10


In [20]:
with employee_hierarchy as(
    select 
        EmployeeID
        , FirstName 
        , ManagerID 
        , 1 as level
        , FirstName as manager
    from 
        Sales.Employees
    where ManagerID is null

    union all 

    select 
          e.EmployeeID
        , e.FirstName 
        , e.ManagerID 
        , level+1 as level
        , coalesce(ceh.FirstName,'Manager') as manages_name
    from 
        Sales.Employees as e
    inner join employee_hierarchy ceh on e.ManagerID = ceh.EmployeeID
)
select * from employee_hierarchy


(5 rows affected)

Total execution time: 00:00:00.019

EmployeeID,FirstName,ManagerID,level,manager
1,Frank,NULL,1,Frank
2,Kevin,1,2,Frank
3,Mary,1,2,Frank
5,Carol,3,3,Mary
4,Michael,2,3,Kevin
